In [121]:
import pandas as pd
import numpy as np
from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper
)
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import *
import shapefile
from bokeh.models import *
from bokeh.models import HoverTool
from collections import OrderedDict

In [122]:
unemployment = pd.read_csv('data/unemployment.csv', encoding='utf-8')

In [193]:
abandoned_buildings = pd.read_csv('data/Abandoned_Buildings_by_ComArea.csv', encoding='utf-8')

In [203]:
abandoned_buildings.head(5)

,Community_Area,Community_Area_Name,Abandoned_Buildings,Latitude,Longitude
0,1,ROGERS PARK,84,42.018652,-87.669745
1,2,WEST RIDGE,208,42.011092,-87.686437
2,3,UPTOWN,122,41.973565,-87.669563
3,4,LINCOLN SQUARE,141,41.973014,-87.689218
4,5,NORTH CENTER,144,41.940480,-87.678846


In [147]:
chicago = shapefile.Reader("Chi_Shapefiles/geo_export_8fdf40c2-dd7b-4f8f-8d9f-d0089dd07307")

In [148]:
com_areas= set(i[4] for i in records)
len(com_areas)


77

In [149]:
records = chicago.records()
fields =  chicago.fields
shapes = chicago.shapes()

In [152]:
fields

[('DeletionFlag', 'C', 1, 0),
 ['perimeter', 'N', 33, 31],
 ['community', 'C', 254, 0],
 ['shape_len', 'N', 33, 31],
 ['shape_area', 'N', 33, 31],
 ['area', 'N', 33, 31],
 ['comarea', 'N', 33, 31],
 ['area_numbe', 'C', 254, 0],
 ['area_num_1', 'C', 254, 0],
 ['comarea_id', 'N', 33, 31]]

In [150]:
shape_records = chicago.shapeRecords()

In [154]:
shape_records[0].record

[0.0, 'DOUGLAS', 31027.0545098, 46004621.1581, 0.0, 0.0, '35', '35', 0.0]

In [197]:
unemployment.rename(columns={'COMMUNITY AREA NAME':'COMMUNITY_AREA_NAME'},inplace=True)
unemployment.rename(columns={'PERCENT AGED 16+ UNEMPLOYED':'Unemployment'},inplace=True)
unemployment['COMMUNITY_AREA_NAME'] = unemployment['COMMUNITY_AREA_NAME'].str.upper()
abandoned_buildings['Community_Area_Name'] = abandoned_buildings['Community_Area_Name'].str.upper()

In [165]:
unemployment.rename(columns={'Community Area Number':'Area_Number'},inplace=True)

In [199]:
unemply = dict(zip(unemployment.COMMUNITY_AREA_NAME,unemployment.Unemployment ))
area_num = dict(zip(unemployment.COMMUNITY_AREA_NAME,unemployment.Area_Number ))
abandoned = dict(zip(abandoned_buildings.Community_Area_Name,abandoned_buildings.Abandoned_Buildings ))

In [222]:
##Borrowed from http://www.abisen.com/blog/bokeh-maps/
##
def getParts ( shapeObj ):

    points = []

    num_parts = len( shapeObj.parts )
    end = len( shapeObj.points ) - 1
    segments = list( shapeObj.parts ) + [ end ]

    for i in range( num_parts ):
        points.append( shapeObj.points[ segments[i]:segments[i+1] ] )

    return points

In [223]:
##Borrowed from http://www.abisen.com/blog/bokeh-maps/

def getDict ( com_area, shapefile ):

    stateDict = {com_area: {} }

    rec = []
    shp = []
    points = []

    for i in shapefile.shapeRecords( ):

        if i.record[1] == com_area:
            rec.append(i.record)
            shp.append(i.shape)

    total_area = sum( [float(i[3]) for i in rec] ) / (1000*1000)


    for j in shp:
        for i in getParts(j):
            points.append(i)

    lat = []
    lng = []
    for i in points:
        lat.append( [j[0] for j in i] )
        lng.append( [j[1] for j in i] )


    stateDict[com_area]['lat_list'] = lat
    stateDict[com_area]['lng_list'] = lng
    stateDict[com_area]['total_area'] = total_area

    return stateDict

In [224]:
county_x = []
county_y = []
total_area = []
county_names = []
area_number = []
for areas in com_areas:
    data = getDict(areas, chicago)
    lon = data[areas]['lng_list'][0]
    lat = data[areas]['lat_list'][0]
    area = data[areas]['total_area']
    county_x.append(lat)
    county_y.append(lon)
    total_area.append(area)
    county_names.append(areas)

In [225]:
colors = ["#F1EEF6", "#D4B9DA", "#C994C7", "#DF65B0", "#DD1C77", "#980043"]

county_colors = []
#county_names = []
county_rates = []
buildings = []
for county_id in unemply:
    rate = unemply[county_id]
    idx = min(int(rate/2), 5)
    number = area_num[county_id]
    abandon = abandoned[county_id]
    county_colors.append(colors[idx])
    buildings.append(abandon)
    county_rates.append(rate)
    #area_number.append(number)

source = ColumnDataSource(
    data = dict(
        x=county_x,
        y=county_y,
        color=county_colors,
        name=county_names,
        buildings=buildings,
        rate=county_rates,
        total_area = total_area
    )
)

In [226]:
TOOLS="pan,wheel_zoom,box_zoom,reset,hover,save"

p = figure(title="Chicago Unemployment", tools=TOOLS)

p.patches('x', 'y', source=source,
          fill_color='color', fill_alpha=0.7,
          line_color="white", line_width=0.5)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    ("Total_Area","@total_area"),
    ("Abandoned Buildings", "@buildings"),
    ("Unemployment rate)", "@rate%"),
    ("(Long, Lat)", "($x, $y)"),
]

output_file("unemployment.html", title="Chicago_Unemployment")

show(p)

INFO:bokeh.core.state:Session output file 'unemployment.html' already exists, will be overwritten.
